# 🏔 Monte Carlo Every Visit

**Monte Carlo Every-Visit** é um algoritmo de controle por Monte Carlo, ou seja, ele estima nossa função de valor *q(s, a)* a partir dos retornos médios de cada par estado-ação, e toma ações no ambiente com base nessas estimativas. 

Entretanto, esse algoritmo difere de outros métodos de Monte Carlo por utilizar todos os retornos de um par estado-ação durante um episódio. Isso significa que, quando o nosso agente visita um estado repetido e toma uma mesma ação, o cálculo da função de valor levará em conta o retorno de todas as vezes que essa ação foi tomada.

## Política ε-greedy

Para garantir que os métodos de Monte Carlo convirjam para a função de valor real, é necessário seguir uma política que explore todas as ações de todos os estados. Entretanto, também é interessante que o agente tente conseguir cada vez mais recompensas, para maximizar sua perfomance.

Assim foi desenvolvida a política *ε-greedy*, que escolhe a próxima ação com base em um parâmetro *ε*, normalmente pequeno. A cada decisão, a política tem uma probabilidade *ε* de escolher uma ação aleatória, aumentando a exploração, e uma probabilidade *1 - ε* de escolher a ação associada ao maior *Q*. Dessa forma, ela estabelece um equilíbrio entre a exploração de ações e a explotação de recompensas. Essa política é dada por:

$$\pi(a|S_t) \leftarrow \begin{cases} 1 - \varepsilon + \varepsilon/\left|\mathcal{A}(S_t)\right|, & \mbox{se } a = \underset{a}{\mathrm{argmax}} \, Q(S_t,a) \\ \varepsilon/\left|\mathcal{A}(S_t)\right|, & \mbox{se } a \neq \underset{a}{\mathrm{argmax}} \, Q(S_t,a) \end{cases}$$

$$\left|\mathcal{A}(S_t)\right| \rightarrow \textrm{quantidade de ações possíveis}$$

## Algoritmo

Primeiramente, devemos inicializar a nossa tabela *Q(s, a)* com valores arbitrários para cada par estado-ação. Nesse caso, vamos optar por superestimar os valores Q de modo a incentivar a exploração do agente.

Também inicializamos uma tabela *N(s, a)* que guarda a quantidade de retornos obtidos de cada par estado-ação, para fazer o cálculo da média móvel dos retornos.

Para cada episódio, vamos escolher ações seguindo nossa política ε-greedy e guardar os estados, ações e recompensas para cada instante *t*. Ao final, calculamos o retorno *G* de cada instante começando pelo término atualizando os valores *Q* correspondentes.

Para estimar a média dos retornos, podemos utilizar a *média móvel*, de forma a realizar os cálculos na hora sem precisar guardar uma lista com todos os retornos. Ao invés disso, precisamos guardar apenas a média anterior e a quantidade total de elementos *n*.

$${\overline {x}}_{novo} = \frac{(n - 1){\overline {x}}_{anterior} + x_n}{n}$$

Por fim, podemos ver abaixo um exemplo em pseudo-código do funcionamento do algoritmo de Monte Carlo Every-Visit:

![On-policy every-visit MC control](imgs/MC.png)

## Código

Antes de começar a programar nosso algoritmo, devemos importar a biblioteca ***gym*** e criar o ambiente *FrozenLake-v0*, que usaremos para testar o Monte Carlo Every-Visit.

In [1]:
import gym

env = gym.make("FrozenLake-v0")
env.seed(42)

[42]

Em seguida, vamos definir uma função argmax, que retornará o índice do elemento de maior valor dentro de um vetor. Usaremos essa função para escolher a ação com maior valor *Q* dentro da nossa tabela de valores.

É importante definir nossa própria função de argmax pois, em casos de empate, ela deve decidir aleatoriamente entre um dos índices dos elementos de maior valor.

In [2]:
def argmax(array):
    return np.random.choice(np.flatnonzero(array == array.max()))

Agora sim podemos criar o nosso agente, uma classe com os métodos *step*, *store_experience* e *update*:
  - `step(self, state, epsilon=0)` - Escolhe a próxima ação a se tomar com base no estado atual.
  
  - `store_experience(self, state, action, reward)` - Guarda a tupla (*estado*, *ação, *recompensa*) atual, para depois realizar o cálculo dos retornos.
  
  - `update(self)` - Atualiza a função valor com base nos retornos do episódio.

In [3]:
import numpy as np
from collections import defaultdict

class MonteCarloAgent(object):
    def __init__(self, gamma, action_space):
        self.q_values = defaultdict(lambda: np.ones(action_space.n))
        self.times_visited = defaultdict(lambda: np.zeros(action_space.n))
        self.experiences = []
        self.gamma = gamma
        self.action_space = action_space
        
    def step(self, state, epsilon=0):
        if np.random.random() < epsilon:
            action = self.action_space.sample()
        else:
            action = argmax(self.q_values[state])
        return action
    
    def store_experience(self, state, action, reward):
        self.experiences.append((state, action, reward))
        
    def update(self):
        g = 0
        for state, action, reward in reversed(self.experiences):
            g = self.gamma*g + reward
            self.times_visited[state][action] += 1
            self.q_values[state][action] = ((self.times_visited[state][action]-1) * self.q_values[state][action] + g)/self.times_visited[state][action]
            
        self.experiences = []

In [4]:
agent = MonteCarloAgent(0.9, env.action_space)

In [5]:
from collections import deque

returns = deque(maxlen=1000)

for episode in range(1, 100001):
    state = env.reset()
    done = False
    
    ep_return = 0
    
    while not done:
        action = agent.step(state, epsilon=0.1)
        next_state, reward, done, _ = env.step(action)
        agent.store_experience(state, action, reward)
        state = next_state
        ep_return += reward
        
    returns.append(ep_return)
    agent.update()
    
    if episode % 500 == 0:
        print(f"Episode: {episode:5d} Success Rate: {np.mean(returns):5.4f}\r", end="")

In [6]:
from collections import deque

returns = deque(maxlen=1000)

for episode in range(1, 1001):
    state = env.reset()
    done = False
    
    ret = 0
    
    while not done:
        action = agent.step(state, epsilon=0)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        ret += reward
        
    returns.append(ret)
    
print(f"Episode: {episode:5d} Success Rate: {np.mean(returns):5.4f}\r", end="")